# Running MD with LAMMPS

In this notebook, we will use LAMMPS (https://lammps.org/) to run a Molecular
Dynamics simulation, using the exact same potential we just exported and used
from ASE.

You'll need to install a version of LAMMPS that can use metatomic models. There are multiple ways to do this, documented [here](https://docs.metatensor.org/metatomic/latest/engines/lammps.html), but the simplest one is to use `conda` to download a pre-compiled version:

```bash
conda create -n lammps-metatomic-env
conda activate lammps-metatomic-env
conda install -c metatensor -c conda-forge lammps-metatomic

# check that the installation worked
lmp -h # should contain metatomic in the "known pair_style" section
```

This demonstrates how metatensor models can be used from multiple MD engines in
the same way. Additionally, this shows that while the models are defined using
Python, they can be loaded and used from a C++ simulation engine through
TorchScript.

In [ ]:
import os
import subprocess

import chemiscope
import chemfiles


| ![TASK](img/clipboard.png) | Change the path below to the path of your LAMMPS installation including `pair_style metatomic`. |
|----------------------------|-------------------------------------------------------------------------------------------------|


In [ ]:
LAMMPS_EXE = ".../bin/lmp"

if not os.path.exists(LAMMPS_EXE):
    raise Exception("Please set the path to LAMMPS")

output = subprocess.run([LAMMPS_EXE, "-h"], stdout=subprocess.PIPE, check=True, encoding="utf8")
if "metatomic" not in output.stdout:
    raise Exception("This installation of LAMMPS does not support pair_style metatomic")

To run MD with LAMMPS, we'll need two input files: one defining the initial
conformation of the atoms and another one defining the simulation settings.

In [ ]:
# Write the input structure using ase
frame = chemfiles.Trajectory("propenol_conformers_dftb.xyz").read()
frame.cell = chemfiles.UnitCell([12, 12, 12])

os.makedirs("lammps-simulation", exist_ok=True)
out = chemfiles.Trajectory(os.path.join("lammps-simulation", "propenol.lmp"), "w", "LAMMPS Data")
out.write(frame)
out.close()

with open(os.path.join("lammps-simulation", "propenol.lmp")) as fd:
    print(fd.read())

We can use any metatomic model that can output energy with LAMMPS, using [`pair_style metatomic`](https://docs.metatensor.org/metatomic/latest/engines/lammps.html#how-to-use-the-code). The corresponding `pair_coeff` then needs to specify how to transform from the `lammps` atom types (from 1 to N) to the atom types in the model (typically the atomic numbers). This should be specified using 

```
pair_coeff * * <model type for lammps type 1>  <model type for lammps type 2>   ...   <model type for lammps type N>
```

| ![TASK](img/clipboard.png) | In the LAMMPS input file below, set the correct pair_coeff for the data file above |
|----------------------------|------------------------------------------------------------------------------------|

In [ ]:
# Write the LAMMPS input manually

LAMMPS_INPUT = """
# use Angstroms, eV, ps as units
units metal
boundary p p p

# read the data we just wrote with ASE
atom_style full
read_data propenol.lmp

# set atoms masses
mass 1 1.008
mass 2 12.011
mass 3 15.999

# Potential definition, using a custom pair style.
# This will load the model we just exported
pair_style metatomic ../propenol-model-with-lj.pt extensions ../extensions

# map from LAMMPS atoms types to the species in the model
pair_coeff * * ...

timestep 0.001  # 1 fs timestep
fix 1 all nve   # use NVE ensemble integration


# output information every step: temperature, potential energy, total energy
thermo 1
thermo_style custom step temp pe etotal


# Output the trajectory in XYZ format, using actual atom names
# instead of LAMMPS numeric types
dump 1 all xyz 1 trajectory.xyz
dump_modify 1 element C H O

# Run the simulation for 200 steps
run 200
"""

with open(os.path.join("lammps-simulation", "run.in"), "w") as fd:
    fd.write(LAMMPS_INPUT)


We can now start the simulation in a sub-process, running LAMMPS with the input
file we defined above.

In [ ]:
subprocess.run(
    [LAMMPS_EXE, "-in", "run.in"],
    cwd="lammps-simulation",
    check=True,
)


Let's visualize the trajectory!

In [ ]:
import ase.io

trajectory = ase.io.read(os.path.join("lammps-simulation", "trajectory.xyz"), ":", format="xyz")

chemiscope.show(
    trajectory, mode="structure", settings={"structure": [{"playbackDelay": 50}]}
)
